# LLM fine tuning

Welcome to this notebook on LLM fine tuning. This notebook guides you through the process of converting a dataset from Huggingface to the `chatml` format and fine tuning an LLM using the LoRA fine tuning technique and the Unsloth training framework.

Specifically, we will use synthetic (i.e. LLM-generated) data consisting of dialogs and a corresponding summary for each dialog to fine tune a small language model called SmolLM2-135M. The purpose is to create an efficient language model that is an expert in summarizing dialog. You can swap the model for any model in the Smollm2 family, including the fine tuned ones.

You might wonder: "Why fine tune a small model instead of simply using a large model that can perform the task out of the box?" The answer to this question lies in the efficiency of the small, fine tuned model. It takes up much less VRAM than a large model, and is thus cheaper to run. It also leaves more VRAM for larger, general models or for other specialized models.

This notebook assumes familiarity with concepts relating to training machine learning models (eg training data vs evaluation data; hyperparameters etc), so I will not dwelve on these concepts. Instead, I will focus on explaining the LoRA fine tuning technique and give a basic intro to the matrix math behind it.

The notebook is structured as follows: First, we'll do some data pre-processing; then we will configure LoRA and finally start the fine tuning process.

The notebook is based on work by Maxime Labonne, e.g. his [blog post](https://mlabonne.github.io/blog/posts/2024-07-29_Finetune_Llama31.html) on LLM fine tuning, on [Unsloth's notebooks](https://github.com/unslothai/unsloth) and on the resources in the [Smollm GitHub repository](https://github.com/huggingface/smollm), but I have added more context where I myself felt a little lost when I first started learning about LLM fine tuning and LoRA.

In [1]:
!pip install --upgrade torch
!pip install datasets
!pip install unsloth
!pip install trl
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 6.1 MB/s eta 0:00:00
 

# Data pre-processing and chat templates
Let's first consider what data for LLM fine tuning typically looks like.

A common dataset "style" is what people sometimes refer to as the "Hugginface style". Below is a toy example of what such a dataset looks like. Each entry is a list of dictionaries. Each dictionary must have two fields: `content` and `role`. The `content` field contains the text input to the LLM. The `role` fields tells the LLM what type of input it is - is it a system prompt (`system`), a user prompt (`user`) or one of its own (previous) answers (`assistant`).  

When read into a Huggingface dataset using `Dataset.from_dict()`, the below example will have a single column called "messages" with two entries:


In [1]:
from datasets import Dataset

dataset = {
    "messages": [
        [ { "content": "You are an expert in summarizing texts. Extract and present the main key point of the input text in one short sentence, including essential details like dates, locations, persons and organizations if necessary.", "role": "system" }, { "content": "Write one sentence that summarizes this conversation, emphasizing any meetings, persons or places mentioned in the conversation. \n\n **Conversation:** \n\n So, I'm thinking about trying this new coffee shop downtown. They apparently roast their own beans, and they have a crazy amount of different brewing methods. I was wondering if you'd be interested in going with me sometime this week.I might be free on Thursday afternoon, maybe around 3 p.m.Thursday works for me. I've been wanting to check out that place too.Perfect. How about we meet at the coffee shop directly?Yeah, that sounds good. What do you think about trying something different, like the siphon brewing method?I'm down for anything.Awesome. I heard they also have some really good pastries, so maybe we can grab a treat while we're there.Definitely.Okay, so Thursday at 3 p.m. at the new coffee shop downtown. Sounds good.Oh, and by the way, I was talking to Sarah, and she mentioned she's looking for a good graphic designer for a project she's working on.Really?Yeah, she said it's for a website redesign. That's interesting. I might be able to help her out. I've been doing some freelance design work lately.That's great. Sarah's got a good budget for this project, and she's really easy to work with. I'll put you two in touch.Thanks, I'd love to hear more about it.No problem. Maybe we can talk about it more at the coffee shop on Thursday.Sounds like a plan.Oh, and while we're on the topic of trades, I was thinking maybe you could help me with something too.Sure, what do you need?Remember how I was telling you about that website I'm building for my photography? I could really use some help with the coding. I'm not the best coder, but I know the basics. What kind of help do you need specifically?Well, I'm having trouble with the navigation menu. It's not displaying correctly on mobile devices.Let me take a look at it. Maybe I can figure it out. That would be amazing.Okay, so it's a deal then. You'll connect me with Sarah, and I'll help you with your website.Perfect.Great. I'm looking forward to our coffee date.Me too.", "role": "user" }, { "content": "The conversation discussed a planned meeting at a new coffee shop downtown on Thursday afternoon to discuss potential trade arrangements, including connecting one party with a graphic design project and providing coding assistance for a website.", "role": "assistant" } ],
        [ { "content": "You are an expert in summarizing texts. Extract and present the main key point of the input text in one short sentence, including essential details like dates, locations, persons and organizations if necessary.", "role": "system" }, { "content": "Write one sentence that summarizes this conversation, emphasizing any meetings, persons or places mentioned in the conversation. \n\n **Conversation:** \n\n I saw a documentary on Bohemian Grove the other day, you know that place in California. It's this massive redwood forest, like seriously huge, and they have this secret society that meets there every year. Apparently, it's been going on for over a hundred years, started by some newspaper guys back in the day. They call themselves the Bohemian Club, and it's supposed to be this exclusive club for powerful people, like politicians, CEOs, and celebrities. Every July, they go up to this grove and have this big retreat where they dress up in togas, build effigies, and burn them. They call it the Cremation of Care ceremony. Sounds a little creepy, right?It's supposed to be a place where they can just relax and let loose, away from the pressures of their everyday lives. But there's a lot of speculation about what really goes on up there. Some people think it's just harmless fun, a bunch of guys getting together for a guys' weekend. Others think it's something more sinister, like a place where they plan world domination or something.There are always protesters outside the grove during the meetings, you know, conspiracy theorists and activists. They accuse the club of being elitist and secretive, and they say they're up to no good. I mean, I don't know what to believe. It's all pretty hush-hush, and they don't let the public in.But the fact that it's so secret just makes people more suspicious. They say there are hidden bunkers and underground tunnels, and that they do all sorts of weird rituals. It's kind of like Area 51, you know? Everyone wants to know what's going on behind closed doors.I read somewhere that Richard Nixon used to go there, and Ronald Reagan too. Apparently, they have a giant owl statue that's supposed to be the symbol of the club. They even have a lake there, and supposedly, they have these little boats shaped like owls that they use to get around.It's just fascinating, you know? This little pocket of secrecy in the middle of the redwoods. It's like stepping back in time, a secret world that most people will never get to see.I think I'd like to see it, just to see what all the fuss is about. Maybe sneak in and take some pictures, but I doubt I could get past the security.", "role": "user" }, { "content": "The conversation discussed the Bohemian Grove, a secluded redwood forest in California where the Bohemian Club, an exclusive society of powerful individuals, meets annually for a secretive retreat featuring the Cremation of Care ceremony.", "role": "assistant" } ]
        ]
    }

dataset = Dataset.from_dict(dataset)
dataset

ModuleNotFoundError: No module named 'datasets'

In order to use a dataset that looks like above for fine tuning, the data has to be pre-processed such that it follow a specific chat template. A base model has not been instruction fine tuned and therefore it has never seen any data structured in any particular chat template. When fine tuning a base model (like we do in this notebook), we can therefore freely choose which chat template to use. After fine tuning, when we use the model in practice, we must stick to our chat template choice. In this guide, we will use the `chatml` template.

We will fine tune the LLM on a concatenation of two datasets:


1.   `ThatsGroes/synthetic-dialog-summaries-processed-clean-chatml`
2.   The `smol-summarize` subset of the `HuggingFaceTB/smoltalk` dataset

I have already converted the former dataset to the `chatml` format, but below, I will show how to convert the latter.

We can convert the dataset to the `chatml` format using `unsloth.chat_template.get_chat_template` like below. `get_chat_template` makes changes to the tokenizer such that it obtains a method called `apply_chat_template` which converts a dataset entry from the Huggingface style we saw above to the specified chat template - in this case `chatml`.

In the `formatting_prompts_func` we add to the dataset a column called `texts` which contains the chatml formatted entries.


In [2]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset, Dataset, concatenate_datasets

def formatting_prompts_func(samples: Dataset, tokenizer, col_to_process: str="messages"):
    """
    Create a new column called "text" by applying the tokenizer's chat template to `col_to_process`
    Args:

        samples: A Huggingface-style dataset
        tokenizer: an Unsloth tokenizer
        col_to_process: The column containing the samples in the dataset. The entries in this column will be converted to the chat template format specified in the tokenizer.
    """
    convos = samples[col_to_process]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.
model_id = "HuggingFaceTB/SmolLM2-135M"
dataset_id1 = "ThatsGroes/synthetic-dialog-summaries-processed-clean-chatml" #"ThatsGroes/synthetic-dialog-summaries-processed-clean"
dataset_id2 = ("HuggingFaceTB/smoltalk", "smol-summarize")
col_to_process = "messages" # the name of the column in the dataset containing the actual training data

### Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id, # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# defaults to using chatml as per https://github.com/unslothai/unsloth/blob/main/unsloth/chat_templates.py
tokenizer = get_chat_template(
    tokenizer
)

# Load dataset1
dataset = load_dataset(dataset_id1) #dataset.map(formatting_prompts_func, batched = True, fn_kwargs={"tokenizer": tokenizer, "col_to_process": col_to_process})

# Load dataset2
smoltalk = load_dataset(dataset_id2[0], dataset_id2[1])

# Convert dataset2 to chatml format using the built in method `map` and our own `formatting_prompts_func` and pass arguments using `fn_kwargs`
smoltalk = smoltalk.map(formatting_prompts_func, batched = True, fn_kwargs={"tokenizer": tokenizer, "col_to_process": col_to_process})

# Concat datasets to training data
training_dataset = concatenate_datasets(
    [
        dataset["train"],
        smoltalk["train"]
    ]
    )

# Create evaluation data
eval_dataset = concatenate_datasets(
    [
        dataset["test"],
        smoltalk["test"]
    ]
    )

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [13]:
training_dataset

DatasetDict({
    train: Dataset({
        features: ['summary', 'dialog', 'system_prompt', 'messages', 'text'],
        num_rows: 949995
    })
    test: Dataset({
        features: ['summary', 'dialog', 'system_prompt', 'messages', 'text'],
        num_rows: 50000
    })
})

In [14]:
eval_dataset

DatasetDict({
    train: Dataset({
        features: ['messages', 'text'],
        num_rows: 96356
    })
    test: Dataset({
        features: ['messages', 'text'],
        num_rows: 5072
    })
})

# LoRA fine tuning

Above, we loaded the model and tokenizer and prepared the dataset. Now, we will fine tune the model on the dataset using Low Rank Adaptation (LoRA) fine tuning. To do so, we must first configure the model for LoRA fine tuning using the `get_peft_model` method. As a side note, the `peft` part of the method's name is short for "parameter efficient fine tuning" which is an umbrella term that denotes a family of techniques for fine tuning models using fewer parameters than full fine tuning. LoRA falls under the PEFT family of fine tuning techniques.

Maxime Labonne gives a gracefully brief description of the LoRA technique:

> "Low-Rank Adaptation (LoRA) is a popular parameter-efficient fine-tuning technique. Instead of retraining the entire model, it freezes the weights and introduces small adapters (low-rank matrices) at each targeted layer. This allows LoRA to train a number of parameters that is drastically lower than full fine-tuning (less than 1%), reducing both memory usage and training time. This method is non-destructive since the original parameters are frozen, and adapters can then be switched or combined at will." (Source: https://mlabonne.github.io/blog/posts/2024-07-29_Finetune_Llama31.html)

Let's dive a little deeper into the matrix math behind LoRA.

LoRA creates an "adaptor matrix" denoted `delta_W` for each layer that you want to target in the original model, denoted `W_original`. Each `W_original` has the following dimensions: 𝑚 × 𝑟 (m rows and n columns).

For each layer, LoRA creates the adaptor matrix `delta_W` = A x B.

- A has dimensions
𝑚
×
𝑟
(a skinny, tall matrix).
- 𝐵 has dimensions
𝑟
×
𝑛
 (a short, wide matrix).
- 𝑟 is much smaller than both
𝑚
 and
𝑛
, which keeps things efficient.

The weights of the new, LoRA fine tuned model is then `W_new  = W_original x delta_W`. During fine tuning, only the weights in `A` and `B` updated.

the "addition" between `W_orginal` and `delta_W` happens element wise
𝑊
original
W
original
​
  and
Δ
𝑊
ΔW happens element-wise. This means that each entry in
𝑊
new
W
new
​
  is the sum of the corresponding entries in
𝑊
original
W
original
​
  and
Δ
𝑊
ΔW.

After you have trained the LoRA adaptors, you can merge them with the original weights. If you do so, the fine tuned model will have the same number of paramters as the original.

## LoRA arguments
As you can see below, Unsloth's LoRA implementation takes many arguments. Here, I will briefly describe those I deem most important.

### `r`
Recall the `r`from before that we used to described the size of the matrices that make up `delta_W` ? That's what the `r` - aka "rank" - argument here controls. So, the rank, `r`, denotes the size of the low-rank matrices. A lower rank means fewer parameters are being learned, leading to a more parameter-efficient method. The rank can be thought of as a trade-off:

- Higher Rank: More capacity to capture complex relationships and potentially better performance, but at the cost of increased parameters and training complexity.

- Lower Rank: Fewer parameters, faster training, and less risk of overfitting, but it may limit the model’s ability to fully adapt to the new task.

In practice, people seem to mostly use either of the following rank values:

`[8, 16, 32, 64, 128, 256]`

It [has been observed](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) that increasing from 8 to 16 did not significantly improve performance , but I have also seen large values used in practice, for instance to fine tune a [Finnish LLM](https://huggingface.co/Finnish-NLP/Ahma-7B-Instruct).

### `target_modules`
This argument determines which layers (aka modules) in the model to train adaptor matrices for. If this is specified, adaptors will only be made for the modules with the specified names. A widespread practice seems to be to target the following modules ([Source](https://mlabonne.github.io/blog/posts/2024-07-29_Finetune_Llama31.html), [Source](https://huggingface.co/Finnish-NLP/Ahma-7B-Instruct)):

`["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]`

### `alpha`
Remember when we said that `W_original = A X B`? Actually, it is
ΔW=
α
​
 (A×B)
 where α is a so-called scaling factor for updates. Alpha directly impacts the contribution of the adators. The larger `alpha` is, the more the LoRA adaptors will dominate the resulting model. If the original models needs a lot of adaptation for the new task that you're fine tuning for, you might want to set `alpha` >>1.  

It seems that in practice, `alpha` is often set to 1x or 2x the rank value. (https://mlabonne.github.io/blog/posts/2024-07-29_Finetune_Llama31.html). It is suggested to not treat the alpha parameter as a tunable hyperparameter (https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2)

### `use_rslora`
RSLoRA means Rank-Stabilized LoRA. I suggest to set this to true when using higher values for rank as per (https://mlabonne.github.io/blog/posts/2024-07-29_Finetune_Llama31.html)


In [3]:
rank = 8
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj"]
use_rslora = True


model = FastLanguageModel.get_peft_model(
    model,
    r = rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128 # A lower rank means fewer parameters are being learned, leading to a more parameter-efficient method
    target_modules = target_modules,
    lora_alpha = rank, # standard practice seems to be to set this to 16. Mlabonne says its usually set to 1-2x the rank
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = use_rslora,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.6 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


# Start training


In [ ]:
import os
import wandb

wandb.login()
os.environ["WANDB_PROJECT"]="llm_dialog_summarizer"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = training_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field = "text", # the key of the dict returned in formatting_prompts_func()
    max_seq_length = max_seq_length,
    dataset_num_proc = 4, # Number of processes to use for processing the dataset. Only used when packing = False
    packing = False, # Can make training 5x faster for short sequences.
    eval_strategy = "steps",
    eval_steps = 100,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2,
        num_train_epochs = 1, # Set this for 1 full training run. OpenAI's default is 4 https://community.openai.com/t/how-many-epochs-for-fine-tunes/7027/5
        learning_rate = 3e-4,
        warmup_steps=100,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear", # linear or cosine are most common
        seed = 90201,
        output_dir = "outputs",
        report_to="wandb",
        run_name = "Baseline",
        save_steps=10000
    ),
)

Map (num_proc=4):   0%|          | 0/949995 [00:00<?, ? examples/s]

In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 949,995 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 59,375
 "-____-"     Number of trainable parameters = 2,442,240
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,3.962200
2,3.692400
3,3.678500
4,3.662100
5,3.960100
6,3.569700
7,3.985500
8,3.583500
9,3.640600
10,3.540300


KeyboardInterrupt: 